In [7]:
import cv2
import numpy as np
from pathlib import Path
import time
import psutil
import os
import gc
import math
from dataclasses import dataclass
from typing import List, Tuple, Optional

In [8]:
class SimplePanorama:
    def __init__(self, initial_frame):
        """初始化全景图系统"""
        self.frame_h, self.frame_w = initial_frame.shape[:2]
        
        # 创建初始画布（给一些边距以便扩展）
        margin = 100
        self.canvas = np.zeros((self.frame_h + 2*margin, self.frame_w + 2*margin, 3), dtype=np.uint8)
        
        # 当前位置（从中心开始）
        self.current_x = margin
        self.current_y = margin
        
        # 记录已使用区域的边界
        self.min_x = margin
        self.max_x = margin + self.frame_w
        self.min_y = margin
        self.max_y = margin + self.frame_h
        
        # 放置第一帧
        self.canvas[margin:margin+self.frame_h, margin:margin+self.frame_w] = initial_frame
    
    def expand_canvas_if_needed(self, new_x, new_y):
        """在需要时扩展画布"""
        need_expand = False
        pad_left = pad_right = pad_top = pad_bottom = 0
        
        # 检查是否需要扩展
        if new_x < 0:
            pad_left = abs(new_x)
            need_expand = True
        if new_x + self.frame_w > self.canvas.shape[1]:
            pad_right = new_x + self.frame_w - self.canvas.shape[1]
            need_expand = True
        if new_y < 0:
            pad_top = abs(new_y)
            need_expand = True
        if new_y + self.frame_h > self.canvas.shape[0]:
            pad_bottom = new_y + self.frame_h - self.canvas.shape[0]
            need_expand = True
            
        if need_expand:
            # 创建新画布
            new_h = self.canvas.shape[0] + pad_top + pad_bottom
            new_w = self.canvas.shape[1] + pad_left + pad_right
            new_canvas = np.zeros((new_h, new_w, 3), dtype=np.uint8)
            
            # 复制原画布内容到新位置
            y_start = pad_top
            x_start = pad_left
            new_canvas[y_start:y_start+self.canvas.shape[0], 
                      x_start:x_start+self.canvas.shape[1]] = self.canvas
            
            # 更新坐标
            self.current_x += pad_left
            self.current_y += pad_top
            self.min_x += pad_left
            self.max_x += pad_left
            self.min_y += pad_top
            self.max_y += pad_top
            
            self.canvas = new_canvas
            return pad_left, pad_top
            
        return 0, 0
    
    def add_frame(self, frame, angle, magnitude):
        """添加新帧到全景图"""
        # 计算新位置（保持之前的正确计算方式）
        dx = magnitude * math.cos(math.radians(angle + 90))
        dy = magnitude * math.sin(math.radians(angle - 90)) # y坐标为负
        
        new_x = int(self.current_x + dx)
        new_y = int(self.current_y + dy)
        
        # 如果需要则扩展画布（保持不变）
        offset_x, offset_y = self.expand_canvas_if_needed(new_x, new_y)
        new_x += offset_x
        new_y += offset_y
        
        # 计算重叠区域
        roi = self.canvas[new_y:new_y+self.frame_h, new_x:new_x+self.frame_w]
        
        # 在重叠区域应用渐变混合
        alpha = np.zeros((self.frame_h, self.frame_w), dtype=np.float32)
        
        # 根据移动方向创建渐变
        if abs(dx) > abs(dy):  # 主要是水平移动
            blend_width = int(abs(dx))  # 使用移动距离作为混合宽度
            if dx > 0:  # 向右移动
                alpha[:, :blend_width] = np.linspace(0, 1, blend_width)
                alpha[:, blend_width:] = 1
            else:  # 向左移动
                alpha[:, -blend_width:] = np.linspace(1, 0, blend_width)
                alpha[:, :-blend_width] = 1
        else:  # 主要是垂直移动
            blend_width = int(abs(dy))  # 使用移动距离作为混合宽度
            if dy > 0:  # 向下移动
                alpha[:blend_width, :] = np.linspace(0, 1, blend_width)[:, np.newaxis]
                alpha[blend_width:, :] = 1
            else:  # 向上移动
                alpha[-blend_width:, :] = np.linspace(1, 0, blend_width)[:, np.newaxis]
                alpha[:-blend_width, :] = 1
        
        # 应用混合
        alpha = np.stack([alpha] * 3, axis=2)
        result = (1 - alpha) * roi + alpha * frame
        
        # 更新画布
        self.canvas[new_y:new_y+self.frame_h, new_x:new_x+self.frame_w] = result.astype(np.uint8)
        
        # 更新位置信息（保持不变）
        self.current_x = new_x
        self.current_y = new_y
        self.min_x = min(self.min_x, new_x)
        self.max_x = max(self.max_x, new_x + self.frame_w)
        self.min_y = min(self.min_y, new_y)
        self.max_y = max(self.max_y, new_y + self.frame_h)
    
    def get_result(self):
        """获取最终结果"""
        # 只返回包含图像的区域
        return self.canvas[self.min_y:self.max_y, self.min_x:self.max_x]

def calculate_movement(img1, img2):
    """计算两帧之间的运动方向和幅度"""
    # 获取图像尺寸用于调试信息
    h, w = img1.shape[:2]
    
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)
    
    print(f"\n== 调试信息 ==")
    print(f"图像尺寸: {w}x{h}")
    print(f"检测到的特征点数量: 帧1={len(keypoints1)}, 帧2={len(keypoints2)}")
    
    if descriptors1 is None or descriptors2 is None:
        print("未检测到特征点")
        return None, None
    
    # 特征匹配
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    
    # 收集特征点位置信息用于分析分布
    frame1_points = []
    frame2_points = []
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
            # 记录匹配点的坐标
            pt1 = keypoints1[m.queryIdx].pt
            pt2 = keypoints2[m.trainIdx].pt
            frame1_points.append(pt1)
            frame2_points.append(pt2)
    
    print(f"良好匹配点数量: {len(good_matches)}")
            
    if len(good_matches) < 10:
        print("匹配点数量不足")
        return None, None
        
    # 计算特征点分布
    frame1_points = np.array(frame1_points)
    frame2_points = np.array(frame2_points)
    
    # 计算特征点的分布统计
    if len(frame1_points) > 0:
        x_min = np.min(frame1_points[:, 0])
        x_max = np.max(frame1_points[:, 0])
        y_min = np.min(frame1_points[:, 1])
        y_max = np.max(frame1_points[:, 1])
        print(f"特征点分布范围: X=({x_min:.1f}, {x_max:.1f}), Y=({y_min:.1f}, {y_max:.1f})")
    
    # 计算平均移动向量
    movements = []
    for match in good_matches:
        pt1 = np.array(keypoints1[match.queryIdx].pt)
        pt2 = np.array(keypoints2[match.trainIdx].pt)
        movement = pt2 - pt1
        movements.append(movement)
    
    mean_movement = np.mean(movements, axis=0)
    dx, dy = mean_movement
    
    # 计算移动统计信息
    movements = np.array(movements)
    std_dev = np.std(movements, axis=0)
    print(f"平均移动: dx={dx:.2f}, dy={dy:.2f}")
    print(f"移动标准差: dx_std={std_dev[0]:.2f}, dy_std={std_dev[1]:.2f}")
    
    # 计算角度（0度为正上方，顺时针旋转）
    angle = math.degrees(math.atan2(-dy, dx))  # 使用-dy是因为图像坐标系y轴向下
    angle = (angle + 90) % 360
    
    # 计算移动幅度
    magnitude = math.sqrt(dx*dx + dy*dy)
    
    # 判断主要移动方向并调整幅度
    is_horizontal = abs(dx) > abs(dy)
    
    # 输出移动方向相关信息
    move_direction = "横向" if is_horizontal else "纵向"
    print(f"主要移动方向: {move_direction}")
    print(f"移动角度: {angle:.2f}°")
    print(f"移动幅度: {magnitude:.2f}")
    print("================\n")
    
    return angle, magnitude


In [ ]:

@dataclass
class PerformanceStats:
    """性能统计数据类"""
    total_time: float
    avg_frame_time: float
    max_memory: float
    avg_memory: float
    total_frames: int
    memory_readings: List[float]

class PerformanceMonitor:
    """性能监控类"""
    def __init__(self):
        self.start_time = time.time()
        self.process_end_time = None  # 添加处理结束时间
        self.max_memory = 0
        self.memory_readings = []
        self.frame_count = 0

    def stop_timer(self):
        """停止计时"""
        self.process_end_time = time.time()

    def update(self) -> None:
        """更新性能监控数据"""
        current_memory = self._get_memory_usage()
        self.memory_readings.append(current_memory)
        self.max_memory = max(self.max_memory, current_memory)
        self.frame_count += 1

    def get_stats(self) -> PerformanceStats:
        """获取性能统计数据"""
        if self.process_end_time is None:
            self.stop_timer()
        total_time = self.process_end_time - self.start_time
        avg_memory = sum(self.memory_readings) / len(self.memory_readings) if self.memory_readings else 0
        
        return PerformanceStats(
            total_time=total_time,
            avg_frame_time=total_time/self.frame_count if self.frame_count else 0,
            max_memory=self.max_memory,
            avg_memory=avg_memory,
            total_frames=self.frame_count,
            memory_readings=self.memory_readings
        )

    @staticmethod
    def _get_memory_usage() -> float:
        """获取当前进程的内存使用量（MB）"""
        process = psutil.Process(os.getpid())
        return process.memory_info().rss / 1024 / 1024

    @staticmethod
    def print_stats(stats: PerformanceStats) -> None:
        """打印性能统计信息"""
        print(f"\n=== 性能统计 ===")
        print(f"总处理时间: {stats.total_time:.2f} 秒")
        print(f"平均每帧处理时间: {stats.avg_frame_time:.2f} 秒")
        print(f"最大内存使用: {stats.max_memory:.2f} MB")
        print(f"平均内存使用: {stats.avg_memory:.2f} MB")
        print(f"总处理帧数: {stats.total_frames}")
        print("================")

class DisplayManager:
    """显示管理类"""
    def __init__(self, max_width: int = 1920, max_height: int = 1080):
        self.MAX_DISPLAY_WIDTH = max_width
        self.MAX_DISPLAY_HEIGHT = max_height
        cv2.namedWindow('Panorama Progress', cv2.WINDOW_NORMAL)
        cv2.namedWindow('Final Panorama', cv2.WINDOW_NORMAL)

    def resize_to_screen(self, image: np.ndarray) -> Tuple[np.ndarray, int, int]:
        """调整图像大小以适应屏幕"""
        height, width = image.shape[:2]
        width_ratio = self.MAX_DISPLAY_WIDTH / width
        height_ratio = self.MAX_DISPLAY_HEIGHT / height
        scale = min(width_ratio, height_ratio, 1.0)

        if scale < 1.0:
            new_width = int(width * scale)
            new_height = int(height * scale)
            resized = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
            return resized, new_width, new_height
        return image, width, height

    def show_progress(self, image: np.ndarray) -> None:
        """显示处理进度"""
        display_result, win_width, win_height = self.resize_to_screen(image)
        cv2.resizeWindow('Panorama Progress', win_width, win_height)
        cv2.imshow('Panorama Progress', display_result)

    def show_final_result(self, image: np.ndarray) -> None:
        """显示最终结果"""
        display_result, win_width, win_height = self.resize_to_screen(image)
        cv2.resizeWindow('Final Panorama', win_width, win_height)
        cv2.imshow('Final Panorama', display_result)

    def cleanup(self) -> None:
        """清理显示资源"""
        cv2.destroyAllWindows()

class VideoProcessor:
    """视频处理类"""
    def __init__(self, video_path: str):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            raise ValueError("无法打开视频文件")

    def set_start_frame(self, start_frame: int) -> None:
        """设置起始帧"""
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    def read_frame(self, skip_frames: int = 0) -> Tuple[bool, Optional[np.ndarray]]:
        """读取帧"""
        # 跳过指定数量的帧
        for _ in range(skip_frames):
            ret = self.cap.grab()
            if not ret:
                return False, None

        # 读取当前帧
        ret, frame = self.cap.read()
        return ret, frame if ret else None

    def cleanup(self) -> None:
        """清理视频资源"""
        if self.cap is not None:
            self.cap.release()

def main(video_path: str, frame_interval: int, start_frame: int = 1):
    """主函数
    Args:
        video_path: 视频文件路径
        frame_interval: 处理帧间隔
        start_frame: 起始帧位置（默认为1）
    """
    OUTPUT_DIR = "output"

    # 创建输出目录
    Path(OUTPUT_DIR).mkdir(exist_ok=True)

    # 初始化各个模块
    perf_monitor = PerformanceMonitor()
    display_manager = DisplayManager()
    video_processor = VideoProcessor(video_path)

    try:
        # 设置起始帧
        video_processor.set_start_frame(start_frame)

        # 读取第一帧
        ret, prev_frame = video_processor.read_frame()
        if not ret:
            raise ValueError("无法读取第一帧")

        # 初始化全景图
        panorama = SimplePanorama(prev_frame)

        # 显示初始状态
        display_manager.show_progress(panorama.get_result())

        while True:
            # 检查是否按下 'q' 键退出
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("\n用户终止处理")
                break

            # 读取当前帧
            ret, curr_frame = video_processor.read_frame(frame_interval - 1)
            if not ret:
                break

            # 计算运动
            angle, magnitude = calculate_movement(prev_frame, curr_frame)
            if angle is None or magnitude is None:
                print("\n无法计算帧间运动，跳过当前帧")
                continue

            # 添加到全景图
            try:
                panorama.add_frame(curr_frame, angle, magnitude)
                print(f"\r处理第 {perf_monitor.frame_count} 帧 - 方向: {angle:.1f}°, 幅度: {magnitude:.1f}", end="")
                display_manager.show_progress(panorama.get_result())
            except Exception as e:
                print(f"\n处理帧时出错: {str(e)}")
                break

            # 更新性能监控
            perf_monitor.update()

            # 更新前一帧
            prev_frame = curr_frame.copy()

            # 定期清理内存
            if perf_monitor.frame_count % 100 == 0:
                gc.collect()

        # 停止性能计时并保存结果
        perf_monitor.stop_timer()
        result = panorama.get_result()
        cv2.imwrite(f'{OUTPUT_DIR}/panorama.jpg', result)
        print(f"\n处理完成，共处理 {perf_monitor.frame_count} 帧")

        # 显示最终结果和性能统计
        display_manager.show_final_result(result)
        print("\n按任意键关闭窗口...")
        cv2.waitKey(0)

        # 打印性能统计
        stats = perf_monitor.get_stats()
        PerformanceMonitor.print_stats(stats)

    finally:
        # 清理资源
        video_processor.cleanup()
        display_manager.cleanup()
        gc.collect()



In [ ]:
if __name__ == "__main__":
    video_path = "video/4.mp4"
    frame_interval = 10
    main(video_path,frame_interval)